In [1]:
!pip install cvxpylayers

In [1]:
import torch
import numpy as np
# import diffcp
import time
from dataclasses import dataclass
from typing import Any
import cvxpy as cp
import tracemalloc
import os
import linecache
from scipy.linalg import block_diag
from cvxpylayers.torch import CvxpyLayer
import cvxpy as cp

In [2]:
n = 100
n_eq_constraints   = 0
n_ineq_constraints = 50
Q = torch.eye(n)
q = torch.rand(n)
q.requires_grad_(True)
A = torch.empty(n_eq_constraints, n)
b = torch.empty(n_eq_constraints)
G = torch.randn(n_ineq_constraints, n)
h = torch.randn(n_ineq_constraints)

optimizer = torch.optim.SGD([q], lr=0.1)

Q_cp = cp.Parameter((n, n), PSD=True)
q_cp = cp.Parameter(n)
# A_cp = cp.Parameter((n_eq_constraints, n))
# b_cp = cp.Parameter(n_eq_constraints)
G_cp = cp.Parameter((n_ineq_constraints, n))
h_cp = cp.Parameter(n_ineq_constraints)
z_cp = cp.Variable(n)

objective_fn = 0.5 * cp.sum_squares(Q_cp @ z_cp) + q_cp.T @ z_cp
constraints = [G_cp @ z_cp <= h_cp, cp.SOC(1.0, z_cp)]

print('calling cvxpylaer...')
problem = cp.Problem(cp.Minimize(objective_fn), constraints)

layer = CvxpyLayer(problem, parameters=[Q_cp, q_cp, G_cp, h_cp], variables=[z_cp])
sol = layer(Q, q, G, h)
z_sol = sol[0]

loss = torch.sum(z_sol)
loss.backward()

cvxpylayer_grad = q.grad.clone().detach()
optimizer.zero_grad()


calling cvxpylaer...


/nethome/zzhao628/anaconda3/envs/bilevel/lib/python3.10/site-packages/cvxpy/reductions/solvers/solving_chain.py:250: UserWarning: Your problem has too many parameters for efficient DPP compilation. We suggest setting 'ignore_dpp = True'.
  warnings.warn(


In [3]:
cvxpylayer_grad

tensor([-0.0375, -0.0358, -0.0563, -0.0548,  0.0462, -0.0217, -0.0245, -0.1209,
        -0.0541, -0.0099, -0.0824, -0.0141, -0.0079,  0.0176,  0.0646, -0.0501,
         0.0656, -0.0412, -0.0070, -0.0167,  0.0143, -0.0508,  0.0386, -0.1093,
        -0.0339, -0.1238,  0.0205, -0.1061,  0.0659,  0.0408, -0.0688, -0.1593,
        -0.0671, -0.0114,  0.0269,  0.0091,  0.0606, -0.0604, -0.0300, -0.0310,
        -0.0034, -0.1573, -0.0326,  0.0314, -0.0685,  0.0401,  0.0324, -0.0689,
        -0.1761, -0.1082,  0.0076, -0.1423,  0.0405, -0.0622,  0.0173, -0.1118,
        -0.0609, -0.0268,  0.0265, -0.0220,  0.0359,  0.0533, -0.0553, -0.1044,
        -0.0928, -0.0844, -0.0735,  0.0322,  0.0183, -0.0126, -0.0448, -0.0303,
        -0.0566, -0.0995, -0.1019,  0.0218,  0.0446,  0.0028,  0.1019,  0.0014,
         0.0585, -0.0132, -0.0317,  0.0385,  0.0290, -0.0865,  0.0475, -0.0777,
         0.0005, -0.1018, -0.0552, -0.1039, -0.0605, -0.0040, -0.1524, -0.0430,
         0.0405, -0.0357,  0.0341,  0.03

In [4]:
def forward_single_qsocp(Q, p, G, h, A, b, l2reg=0.0, l2con=True, verbose=False):
    nz, neq, nineq = p.shape[0], A.shape[0] if A is not None else 0, G.shape[0] if G is not None else 0

    z_ = cp.Variable(nz)

    obj = cp.Minimize(
        0.5 * cp.quad_form(z_, Q) + p.T @ z_ + cp.pnorm(z_, p=2) * l2reg)
    eqCon = A @ z_ == b if neq > 0 else None
    if nineq > 0:
        slacks = cp.Variable(nineq)
        ineqCon = G @ z_ + slacks == h
        slacksCon = slacks >= 0
    else:
        ineqCon = slacks = slacksCon = None
    if l2con:
      socCon = cp.SOC(1.0, z_)
    else:
      socCon = None
    cons = [x for x in [eqCon, ineqCon, slacksCon, socCon] if x is not None]
    prob = cp.Problem(obj, cons)
    prob.solve(verbose=verbose) # max_iters=5000)
    # prob.solve()
    # prob.solve(adaptive_rho = False)  # solver=cp.SCS, max_iters=5000, verbose=False)
    # prob.solve(solver=cp.SCS, max_iters=10000, verbose=True)
    assert('optimal' in prob.status)
    zhat = np.array(z_.value).ravel()
    nu = np.array(eqCon.dual_value).ravel() if eqCon is not None else None
    if ineqCon is not None:
        lam = np.array(ineqCon.dual_value).ravel()
        slacks = np.array(slacks.value).ravel()
    else:
        lam = slacks = None
    cdual = socCon.dual_value if socCon is not None else None
    return prob.value, zhat, nu, lam, slacks, cdual

In [5]:
p = q.detach()
_, zhati, nui, lami, si, cdual = forward_single_qsocp(
    *[x.cpu().numpy() if x is not None else None
      for x in (Q, p, G, h, None, None)]
)

In [6]:
np.linalg.norm(zhati - z_sol.detach().cpu().numpy())

0.00011063236022012979

In [7]:
torch.norm(Q @ zhati + p + G.T @ lami - cdual[1][:, 0])

tensor(2.3491e-06, dtype=torch.float64)

In [8]:
c = torch.from_numpy(cdual[1]).T
d = torch.from_numpy(cdual[0])

dual_cutoff = 1e-4
active = (torch.from_numpy(lami) > dual_cutoff).float()
G_active = torch.cat((G * active.unsqueeze(-1), c), dim=0)
h_active = torch.cat((h * active, -d), dim=0)
newzhat = forward_single_qsocp(
    *[x.cpu().numpy() if x is not None else None
      for x in (Q, p, None, None, G_active, h_active)], l2con=False, l2reg=float(d)
)[1]
np.linalg.norm(newzhat - zhati)

3.698058927430977e-07

In [9]:
alpha = 1
newp = p + torch.ones_like(p) / alpha
# theor. correct ffoqp
newzhat = forward_single_qsocp(
    *[x.cpu().numpy() if x is not None else None
      for x in (Q, newp, None, None, G_active, h_active)], l2con=False, l2reg=float(d)
)[1]
ffoqp_grad = torch.from_numpy((newzhat - zhati) * alpha)
print('gradient difference', torch.norm(ffoqp_grad - cvxpylayer_grad, p=1))
print('cosine similarity', torch.nn.functional.cosine_similarity(ffoqp_grad, cvxpylayer_grad, dim=0))


gradient difference tensor(1.2307, dtype=torch.float64)
cosine similarity tensor(1.0000, dtype=torch.float64)


In [10]:
# lpgd
newzhat = forward_single_qsocp(
    *[x.cpu().numpy() if x is not None else None
      for x in (Q, newp, G, h, None, None)]
)[1]
lpgd_grad = torch.from_numpy((newzhat - zhati) * alpha)
print('gradient difference', torch.norm(lpgd_grad - cvxpylayer_grad, p=1))
print('cosine similarity', torch.nn.functional.cosine_similarity(lpgd_grad, cvxpylayer_grad, dim=0))


gradient difference tensor(3.2186, dtype=torch.float64)
cosine similarity tensor(0.9840, dtype=torch.float64)


In [11]:
# ffoqp linCon only
newzhat = forward_single_qsocp(
    *[x.cpu().numpy() if x is not None else None
      for x in (Q, newp, None, None, G_active[:-1], h_active[:-1])]
)[1]
ffoqp_grad = torch.from_numpy((newzhat - zhati) * alpha)
print('gradient difference', torch.norm(ffoqp_grad - cvxpylayer_grad, p=1))
print('cosine similarity', torch.nn.functional.cosine_similarity(ffoqp_grad, cvxpylayer_grad, dim=0))


gradient difference tensor(3.2057, dtype=torch.float64)
cosine similarity tensor(0.9887, dtype=torch.float64)


In [14]:
from ffocp_eq_cone import BLOLayer
blolayer = BLOLayer(problem, parameters=[Q_cp, q_cp, G_cp, h_cp], variables=[z_cp], compute_cos_sim=False)
sol, = blolayer(Q, q, G, h)
z_sol = sol[0]

loss = torch.sum(z_sol)
loss.backward()

ffocp_grad = q.grad.clone().detach()
optimizer.zero_grad()
print('gradient difference', torch.norm(ffocp_grad - cvxpylayer_grad, p=1))
print('cosine similarity', torch.nn.functional.cosine_similarity(ffocp_grad, cvxpylayer_grad, dim=0))

gradient difference tensor(0.0062)
cosine similarity tensor(1.0000)


In [15]:
ffocp_grad

tensor([-0.0376, -0.0358, -0.0563, -0.0548,  0.0462, -0.0216, -0.0245, -0.1208,
        -0.0540, -0.0098, -0.0825, -0.0140, -0.0078,  0.0176,  0.0644, -0.0502,
         0.0655, -0.0413, -0.0070, -0.0167,  0.0144, -0.0507,  0.0385, -0.1093,
        -0.0338, -0.1237,  0.0205, -0.1062,  0.0660,  0.0406, -0.0688, -0.1593,
        -0.0670, -0.0115,  0.0269,  0.0091,  0.0606, -0.0603, -0.0299, -0.0310,
        -0.0033, -0.1572, -0.0326,  0.0314, -0.0684,  0.0401,  0.0325, -0.0690,
        -0.1761, -0.1082,  0.0077, -0.1422,  0.0405, -0.0622,  0.0172, -0.1117,
        -0.0608, -0.0270,  0.0266, -0.0220,  0.0359,  0.0532, -0.0553, -0.1045,
        -0.0929, -0.0844, -0.0734,  0.0322,  0.0184, -0.0127, -0.0448, -0.0302,
        -0.0567, -0.0995, -0.1019,  0.0218,  0.0447,  0.0027,  0.1018,  0.0014,
         0.0586, -0.0133, -0.0318,  0.0385,  0.0291, -0.0864,  0.0475, -0.0776,
         0.0006, -0.1018, -0.0552, -0.1039, -0.0606, -0.0039, -0.1524, -0.0430,
         0.0406, -0.0358,  0.0340,  0.03